In [9]:
import pandas as pd
import PyPDF2
import re
from datetime import datetime

In [10]:
doc = r"C:\Users\Carlo\Downloads\TimeCard Report 11.09.2024.pdf"
with open(doc, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    pdf_text = "".join(page.extract_text() or "" for page in reader.pages)

print(pdf_text)

Contract Employee - H2B Weekly Timecard Report
Includes selected workgroups for all employees who are Hourly and Active.
Attendance Enterprise 2.3.40.1002   11/11/2024 9:23:28 AM Page 1 of 21*AGUILAR, Jacob
Cost Center Green End Step
Supervisor DE1 - Thomas
Job Type XX-Utility
Badge 3098
ID Number 30980000
SSN 000-00-0000
Class Pay Class #2 - No Ben Rate 0.0000
Hire Date 05/14/24
11/03/24 to 11/09/24
Date InOut InOutHours Schedule Exceptions
11/04/24  6:52p  7:01a 12:15  7:00p/ 7:00a
11/05/24  6:53p  7:02a 12:00  7:00p/ 7:00a
11/08/24  6:53a  7:05p 12:00  7:00a/ 7:00p
11/09/24  6:53a  7:05p 12:00  7:00a/ 7:00p
Regular   40:00Overtime  8:15
Supervisor Edits
 BKTHOMAS 11/08/24  7:20p AUTHORIZED  PERIOD BEGINNING 11/03/24
*AGUIRRE, Pedro
Cost Center Green End Step
Supervisor GE2 - Gagnard
Job Type XX-Utility
Badge 3599
ID Number 3599000
SSN 000-00-0000
Class Pay Class #2 - No Ben Rate 0.0000
Hire Date 05/10/24
11/03/24 to 11/09/24
Date InOut InOutHours Schedule Exceptions
11/03/24  6:54a 

In [11]:
separateData = pdf_text.split('*')
separateData.pop(0)
cut = separateData[-1].split('Total')
separateData[-1] = cut[0]
print(separateData[33])

PEREZ SANTIZO, JuniorContract Employee - H2B Weekly Timecard Report
Includes selected workgroups for all employees who are Hourly and Active.
Attendance Enterprise 2.3.40.1002   11/11/2024 9:23:28 AM Page 14 of 21Cost Center Lay-up Line & Press Step
Supervisor LU4 - Lee
Job Type XX-Utility
Badge 3095
ID Number 30950000
SSN 000-00-0000
Class Pay Class #2 - No Ben Rate 0.0000
Hire Date 04/16/24
11/03/24 to 11/09/24
Date InOut InOutHours Schedule Exceptions
11/03/24  6:50p  7:00a 12:15  7:00p/ 7:00a Arr. Early
11/05/24  6:54a  3:00p 8:00 Unsch
11/06/24  6:54a  7:06p 12:00  7:00a/ 7:00p
11/07/24  6:53a  7:03p 12:00  7:00a/ 7:00p
Regular   40:00Overtime  4:15
Supervisor Edits
 RDLEE 11/07/24  6:00p AUTHORIZED  PERIOD BEGINNING 11/03/24



In [12]:
nameList = []
monday = []
tuesday = []
wednesday = []
thursday = []
friday = []
saturday = []
sunday = []
regularTimeList = []
overTimeList = []

In [ ]:
def extractionData(pdf):
    cleanLists()

    for data in pdf:
        dataExtraction(data)    

def cleanLists():
    nameList.clear()
    monday.clear()
    tuesday.clear()
    wednesday.clear()
    thursday.clear()
    friday.clear()
    saturday.clear()
    sunday.clear()
    regularTimeList.clear()
    overTimeList.clear()

def dataExtraction(data):
    name = namesListExtraction(data)
    nameList.append(name)

    regularHours = regularHoursExtraction(data)
    regularTimeList.append(regularHours)

    overHours = overHoursExtraction(data)
    overTimeList.append(overHours)

    timeExtraction(data)

def namesListExtraction(data):
    fullName = str(data).partition("\n")[0]
    if len(fullName) > 40:
        fullName = fullName.split(',')[0].strip()

        fullName = fullName.split(' ')
        name = fullName[1].strip().lower().capitalize().split()
        lastName = fullName[0].lower().capitalize()
        realName = " ".join(name) + " " + lastName
        return realName

    else:
        fullName = fullName.split(',')
        name = fullName[1].strip().lower().capitalize().split()
        lastName = fullName[0].lower().capitalize()
        realName = " ".join(name) + " " + lastName
        return realName

def regularHoursExtraction(data):

    firstCoordinate = re.search('Regular', data)
    secondCoordinate = re.search('Over', data)

    # no every worker has hours worked, they could be in the list but with no time on it. 
    if firstCoordinate and secondCoordinate:
        first = firstCoordinate.span()[1]
        second = secondCoordinate.span()[0]
        
        regularTime = str(data[first:second]).strip().replace(':', '.')
        return regularTime
    
    else:
        return None
    
def overHoursExtraction(data):
    overHours = re.search("Overtime\s+(\d+:\d+)",separateData[-1])
    return overHours.group(1)

def timeExtraction(data):
    pattern = r'(\d{2}/\d{2}/\d{2})\s+(\d{1,2}:\d{2}[apAPmM])\s+(\d{1,2}:\d{2}[apAPmM])\s+(\d{2}:\d{2})'
    dateAndTime = re.findall(pattern, data)

    for index in range(len(dateAndTime)):
        time = str(dateAndTime[index][3])
        date = str(dateAndTime[index][0])
        weekDay = datetime.strptime(date, "%m/%d/%y")
        weekDay = weekDay.strftime("%A") # verify if which week day is the date

        if weekDay == "Monday":
            monday.append(time)
        elif weekDay == "Tuesday":
            tuesday.append(time)
        elif weekDay == "Wednesday":
            wednesday.append(time)
        elif weekDay == "Thursday":
            thursday.append(time)
        elif weekDay == "Friday":
            friday.append(time)
        elif weekDay == "Saturday":
            saturday.append(time)
        elif weekDay == "Sunday":
            sunday.append(time)

In [14]:
extractionData(separateData)

print(nameList, regularTimeList, overTimeList, monday, tuesday, wednesday, thursday, friday, saturday, sunday, sep='\n\n')

['Jacob Aguilar', 'Pedro Aguirre', 'Anthony Alexander', 'Adelaido Alfaro', 'Erasmo Anguiano', 'Jonathan Ariola', 'Alonzo Campbell', 'Jose Cerda', 'Michael Cruz', 'David Diaz', 'Flavio Espinosa', 'Robert Galindo', 'Jose Gallegos', 'Maria Garay', 'Edgar Garcia', 'Cesareo Gonzales', 'Eduardo Gonzalez', 'Joseph Gonzalez', 'Roberto Gonzalez', 'Kenneth Gordon', 'Robert Grant', 'Juan Guerra', 'Malena Guevara-rubio', 'Juan Herrera', 'Christian Ibarra', 'Elijah Lauriano', 'Dyson Lopez', 'Jonathnan l Lopez', 'Jorge Lopez', 'Jason Magana', 'Felix Navarro', 'Luis Ortega', 'Azion Ortiz', 'Santizo Perez', 'Angel Perez', 'Gerardo Perez', 'Dylan Perillo', 'Ricky Prado', 'Joseph Quinones', 'Damian Reyes', 'Eduardo Rios', 'Anthony Rivera', 'Luis ocasio Rodriguez', 'Robert Rodriguez', 'Jason Romero', 'David Rosales', 'Stephen Soto', 'Eric Thomas', 'Brandon Torres', 'Malvin Torres-carrero', 'Daniel Vasquez', 'Brijido Ysquierdo']

['40.00', '40.00', '40.00', '40.00', '40.00', '40.00', '40.00', '40.00', '40

In [15]:
dfNameList = pd.DataFrame(nameList, columns=['name'])
dfRegularTimeList = pd.DataFrame(regularTimeList, columns=['rt'])
dfOvertimeList = pd.DataFrame(overTimeList, columns=['ot'])

# time recording
dfMonday = pd.DataFrame(monday, columns=['monday'])
dfTuesday = pd.DataFrame(tuesday, columns=['tuesday'])
dfWednesday = pd.DataFrame(wednesday, columns=['wednesday'])
dfThursday = pd.DataFrame(thursday, columns=['thursday'])
dfFriday = pd.DataFrame(friday, columns=['friday'])
dfSaturday = pd.DataFrame(saturday, columns=['saturday'])
dfSunday = pd.DataFrame(sunday, columns=['sunday'])

payroll = pd.concat([dfNameList, dfMonday, dfTuesday, dfWednesday, dfThursday, dfFriday, dfSaturday, dfSunday, dfRegularTimeList, dfOvertimeList], axis=1)
payroll = payroll.sort_values('name',ignore_index=True)
payroll = payroll.fillna(0)

print(payroll)

                     name monday tuesday wednesday thursday friday saturday  \
0         Adelaido Alfaro  12:00   12:00     12:00    12:00  12:00    12:00   
1         Alonzo Campbell  12:00   12:00     12:00    12:00  12:00    12:00   
2             Angel Perez      0   12:00         0        0      0        0   
3       Anthony Alexander  12:00   12:00     12:00    12:00  12:15    12:00   
4          Anthony Rivera      0       0         0        0      0        0   
5             Azion Ortiz      0   12:00         0        0  12:00        0   
6          Brandon Torres      0       0         0        0      0        0   
7       Brijido Ysquierdo      0       0         0        0      0        0   
8        Cesareo Gonzales  12:00   12:00     12:00    12:00  12:00    12:15   
9        Christian Ibarra  12:00   12:00     12:00    12:00  12:15    12:00   
10           Damian Reyes      0       0         0        0      0        0   
11         Daniel Vasquez      0       0         0  

In [16]:
payroll.to_csv(r"C:\Users\Carlo\OneDrive\Documents\Python\Docs\payrollPollock.csv",index=False)